In [ ]:
from copy import deepcopy
import subprocess
import socket

import sys

import concurrent.futures

In [ ]:
l = int(input("Horizontal number of wifi-access points: "))
b = int(input("Vertical number of wifi-access points: "))
n = 0
while n % 2 == 0 and n <= 9:
    n = int(
        input("Number of mesh-points between two wifi-access points (odd and <=9): ")
    )
hops = int(input("Max hops: "))

In [ ]:
def n_num_in_list(_list, n):
    ans = []
    def n_num_in_list_util(_l, _n, _ans):
        if _n == 0:
            ans.append(_ans)
            return

        for i in range(len(_l)):
            a = deepcopy(_l)[i+1:]
            # a.pop(i)
            b = deepcopy(_ans)
            b.append(_l[i])
            n_num_in_list_util(
                a,
                _n-1,
                b    
            )

    n_num_in_list_util(_list, n, [])
    
    return ans

In [ ]:
def conn_wifi_mesh(wifi_name, mesh_name):
    ans = False
    if int(mesh_name[-1]) <= int(n / 2):
        ans = ans or wifi_name == mesh_name[:2]
    if int(mesh_name[-1]) >= int(n / 2):
        ans = ans or wifi_name == mesh_name[2:4]
    return ans

def conn_mesh_mesh(m_name1, m_name2):
    fh1 = m_name1[:2]
    fh2 = m_name2[:2]

    lh1 = m_name1[2:4]
    lh2 = m_name2[2:4]

    if fh1 == fh2 and lh1 == lh2:
        return abs(int(m_name1[-1]) - int(m_name2[-1])) == 1

    if fh1 == fh2:
        return m_name1[-1] == 0 and m_name2[-1] == 0

    if fh1 == lh2:
        return m_name1[-1] == 0 and m_name2[-1] == n - 1
    if fh2 == lh1:
        return m_name2[-1] == 0 and m_name1[-1] == n - 1

    return False

In [ ]:
def all_conn_mesh(mesh, key):
    res = set()
    for k in mesh.keys():
        if conn_mesh_mesh(k, key):
            res.add(k)
    return res

In [ ]:
# param = {'00':True, '01':False, ...}
def total_connected_devices_util(wifi={}):
    mesh = {}

    pi, pj = None, None
    for i in range(b):
        for j in range(l):
            if pi != None and pj != None:
                for k in range(n):
                    mesh[str(pi) + str(pj) + str(i) + str(j) + str(k)] = False
            pj = j
            pi = i
        pj = None

    pi, pj = None, None
    for j in range(l):
        for i in range(b):
            if pi != None and pj != None:
                for k in range(n):
                    mesh[str(pi) + str(pj) + str(i) + str(j) + str(k)] = False
            pi = i
            pj = j
        pi = None

    for i in range(b):
        for j in range(l):
            if str(i) + str(j) not in wifi:
                wifi[str(i) + str(j)] = True

    for key in mesh.keys():
        for k in wifi:
            if conn_wifi_mesh(k, key):
                mesh[key] = mesh[key] or wifi[k]

    tmp_keys = set([1])
    for key in mesh.keys():

        def fun(_key, _hops):
            if _hops > 0:
                for k in all_conn_mesh(mesh, _key):
                    tmp_keys.add(k)
                    fun(k, _hops - 1)

        if mesh[key]:
            fun(key, hops)
    for k in tmp_keys:
        mesh[k] = True

    # print(mesh)
    ct = 0
    for k, v in mesh.items():
        if v:
            ct += 1
        # else:
        #     print(v)
    return ct

In [ ]:
n_threads = int(input("Enter number of threads: "))
def _local(off_indices, wifi):
    dic = {}
    for _ in off_indices:
        dic[wifi[_]] = False
    return total_connected_devices_util(dic)

# pass number of wifi to turn off
def total_connected_devices(off_ct):
    wifi = []
    for i in range(b):
        for j in range(l):
            wifi.append(str(i) + str(j))
    
    off_indices_list = n_num_in_list([i for i in range(len(wifi))], off_ct)

    val, ct = 0, len(off_indices_list)

    with concurrent.futures.ProcessPoolExecutor(max_workers=n_threads) as e:
        futures = [e.submit(_local, off_indices, wifi) for off_indices in off_indices_list]
        for f in futures: 
            if f.result():
                a = f.result()
                val+=a
#                 print(a)
    
    return int(val/ct)

In [ ]:
_h = hops
# 0 hops -> only wifi
hops = 0
for ct in range(l*b):
    print(ct, total_connected_devices(off_ct = ct))

In [ ]:
# lets say 5 hops allowed
hops = _h
for ct in range(l*b):
    print(ct, total_connected_devices(off_ct = ct))